## limpieza y procesamiento de tablas

0. Juntar tablas de proyectos y evaluadores (uno a muchos)
1. Aplicar limpieza y steamming
2. Agrupar textos por evaluador
3. Análisis de longitud de textos por investigador

---




In [1]:
import pandas as pd
import pickle

import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

from os import listdir
from math import sqrt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import sent_tokenize, word_tokenize
import heapq

nltk.download('wordnet')
nltk.download('punkt')

stemmer = SnowballStemmer('spanish')

pd.options.mode.chained_assignment = None #...
import seaborn as sns


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/danielbustillos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielbustillos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Descripción del proyecto:

In [2]:
data_topics = pd.read_csv("./data/pn/PN_proyecto_topicmodeling.csv")
data_topics.head(2)

,ID_PROYECTO,NUMERO_CONVOCATORIA,ANIO,USUARIO_RESPONSABLE_TECNICO,NOMBRE_RESPONSABLE_TECNICO,PRIMER_APELLIDO,SEGUNDO_APELLIDO,ES_RESPONSABLE_TECNICO,TITULO,PALABRAS_CLAVE1,...,AREA,DICTAMEN,DESCRIPCION,AUTORIZADO,FECHA_AUTORIZACION,ESTATUS_FORMALIZACION,DESCRIPCION_PROYECTO_FORM,NOMBRE_INSTITUCION,DESCRIPCION_PROYECTO_FASE,DESCRIPCION_PROYECTO
0,1105,PROBLEMAS-2015-01,2015,X_olopez4074,OLIVER XAVIER,LOPEZ,CORONA,NaN,Desarrollo de un Sistema inteligente de soport...,Inteligencia artificial,...,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,La propuesta de investigacion es muy novedosa ...,Desarrollo de un Sistema inteligente de soport...,0.0,NaN,NaN,NaN,UNIVERSIDAD IBEROAMERICANA AC,Etapa inicial del proyecto donde fundamentalme...,El mundo actual es sin duda altamente dinamico...
1,1775,PROBLEMAS-2015-01,2015,X_gaguilar21233,GENARO,AGUILAR,GUTIERREZ,NaN,Politicas publicas para enfrentar la pobreza e...,pobreza extrema,...,CIENCIAS SOCIALES,Es una propuesta pertinente que aportara conoc...,Politicas publicas para enfrentar la pobreza e...,1.0,2016-08-10 15:11:29,FIRMADO,NaN,INSTITUTO POLITECNICO NACIONAL,durante esta primera etapa nos proponemos llev...,POLITICAS PUBLICAS PARA ENFRENTAR LA POBREZA E...


Funciones para estandarizar y limpiar los datos:

In [5]:
def clean_str_series(s):
    """
    info:
    """
    
    
    s = s.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') \
         .str.capitalize().str.strip().str.replace('[^\w\s]','')
    return s

In [6]:
def text_cleaner(df,columns_to_clean,columns_not_na):
    """
    info:
    """
    
    
    # Quitar registros no validos
    df = df.dropna(subset=columns_not_na,axis=0)

    # Formato texto
    for d in columns_to_clean:
        if df[d].dtype == object: 
            df[d] = clean_str_series(df[d])
    
    return df

In [7]:
columns_interes = ["PALABRAS_CLAVE1","PALABRAS_CLAVE2","PALABRAS_CLAVE3",
                    "DESCRIPCION","DESCRIPCION_PROYECTO_FASE","DESCRIPCION_PROYECTO"]
columns_not_na = ["DESCRIPCION_PROYECTO",]

data_topics_clean = text_cleaner(data_topics, columns_interes, columns_not_na)

## Dataset evaluadores:

In [239]:
data_eval = pd.read_csv("./data/pn/PN_proyecto_evaluadores.csv")
data_eval.shape

(8188, 12)

In [240]:
data_eval.columns

Index(['ID_PROYECTO', 'NUMERO_CONVOCATORIA', 'ANIO', 'USUARIO', 'NOMBRE',
       'CVE_RCEA', 'INSTITUCION', 'DEPENDENCIA', 'SUBDEPENDENCIA', 'PAIS',
       'ESTADO', 'CVU'],
      dtype='object')

Número total de evaluadores:

In [10]:
data_eval["CVE_RCEA"].nunique()

3421

## Merge de proyectos y evaluadores

Se duplicarán registros...

In [12]:
data = data_topics_clean.merge(data_eval, on=["ID_PROYECTO","NUMERO_CONVOCATORIA","ANIO"])

#### evaludores por convocatoria o al revles

### Steamming

In [13]:
def stemSentence(sentence, min_len=4):
    """
    info
    """
    
    token_words = word_tokenize(sentence)
    stem_sentence = []
    
    for word in token_words:
        if len(word) > min_len:
            stem_sentence.append(stemmer.stem(WordNetLemmatizer().lemmatize(word, pos='v')))
            stem_sentence.append(" ")
    return "".join(stem_sentence)

obtener raices y quitar palabras cortas

In [14]:
data["DESCRIPCION_PROYECTO"].shape

(8188,)

In [16]:
%%time 
text_data = data["DESCRIPCION_PROYECTO"]
text_data = text_data.apply(stemSentence)

CPU times: user 23min 47s, sys: 12.4 s, total: 23min 59s
Wall time: 24min 55s


In [17]:
data["DESCRIPCION_PROYECTO"] = text_data

In [18]:
data.to_csv("./data/procesado/data_training.csv",index=True)

In [362]:
data = pd.read_csv("./data/procesado/data_training.csv")

In [363]:
data.head(2)

,Unnamed: 0,ID_PROYECTO,NUMERO_CONVOCATORIA,ANIO,USUARIO_RESPONSABLE_TECNICO,NOMBRE_RESPONSABLE_TECNICO,PRIMER_APELLIDO,SEGUNDO_APELLIDO,ES_RESPONSABLE_TECNICO,TITULO,...,DESCRIPCION_PROYECTO,USUARIO,NOMBRE,CVE_RCEA,INSTITUCION,DEPENDENCIA,SUBDEPENDENCIA,PAIS,ESTADO,CVU
0,0,1105,PROBLEMAS-2015-01,2015,X_olopez4074,OLIVER XAVIER,LOPEZ,CORONA,NaN,Desarrollo de un Sistema inteligente de soport...,...,mund actual alt dinam complej condicion socioa...,X_jpena904,JUAN JOSE PE?A CABRIALES,RCEA-06-06773-2002,CENTRO DE INVESTIGACION Y DE ESTUDIOS AVANZADO...,UNIDAD IRAPUATO,DEPARTAMENTO DE BIOTECNOLOGIA Y BIOQUIMICA,MEX,DF,904
1,1,1105,PROBLEMAS-2015-01,2015,X_olopez4074,OLIVER XAVIER,LOPEZ,CORONA,NaN,Desarrollo de un Sistema inteligente de soport...,...,mund actual alt dinam complej condicion socioa...,X_hmorales22194,HELDA ELEONORA DE GUADALUPE MORALES,RCEA-06-11606-2006,EL COLEGIO DE LA FRONTERA SUR,UNIDAD SAN CRISTOBAL DE LAS CASAS,UNIDAD SAN CRISTOBAL DE LAS CASAS,MEXICO,CHIAPAS,22194


In [364]:
data["AREA"]

0       CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...
1       CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...
2       CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...
3                                       CIENCIAS SOCIALES
4                                       CIENCIAS SOCIALES
                              ...                        
8183               CIENCIAS AGROPECUARIAS Y BIOTECNOLOGIA
8184               CIENCIAS AGROPECUARIAS Y BIOTECNOLOGIA
8185               CIENCIAS AGROPECUARIAS Y BIOTECNOLOGIA
8186                HUMANIDADES Y CIENCIAS DE LA CONDUCTA
8187                HUMANIDADES Y CIENCIAS DE LA CONDUCTA
Name: AREA, Length: 8188, dtype: object

In [365]:
data[["ID_PROYECTO","DESCRIPCION_PROYECTO",'CVE_RCEA',"NOMBRE","CVU"]].head(5)

,ID_PROYECTO,DESCRIPCION_PROYECTO,CVE_RCEA,NOMBRE,CVU
0,1105,mund actual alt dinam complej condicion socioa...,RCEA-06-06773-2002,JUAN JOSE PE?A CABRIALES,904
1,1105,mund actual alt dinam complej condicion socioa...,RCEA-06-11606-2006,HELDA ELEONORA DE GUADALUPE MORALES,22194
2,1105,mund actual alt dinam complej condicion socioa...,RCEA-06-17175-2009,ARTURO PEREZ VAZQUEZ,58970
3,1775,polit public enfrent pobrez extrem mediant dis...,RCEA-05-06026-2003,GERARDO MANUEL ORDO?EZ BARBA,11603
4,1775,polit public enfrent pobrez extrem mediant dis...,RCEA-05-06058-2003,FELIX ACOSTA DIAZ,12725


## Juntar textos por evaluador

In [366]:
data.columns

Index(['Unnamed: 0', 'ID_PROYECTO', 'NUMERO_CONVOCATORIA', 'ANIO',
       'USUARIO_RESPONSABLE_TECNICO', 'NOMBRE_RESPONSABLE_TECNICO',
       'PRIMER_APELLIDO', 'SEGUNDO_APELLIDO', 'ES_RESPONSABLE_TECNICO',
       'TITULO', 'PALABRAS_CLAVE1', 'PALABRAS_CLAVE2', 'PALABRAS_CLAVE3',
       'DESCRIPCION_CONVOCATORIA', 'SUBDISCIPLINA', 'DISCIPLINA', 'AREA',
       'DICTAMEN', 'DESCRIPCION', 'AUTORIZADO', 'FECHA_AUTORIZACION',
       'ESTATUS_FORMALIZACION', 'DESCRIPCION_PROYECTO_FORM',
       'NOMBRE_INSTITUCION', 'DESCRIPCION_PROYECTO_FASE',
       'DESCRIPCION_PROYECTO', 'USUARIO', 'NOMBRE', 'CVE_RCEA', 'INSTITUCION',
       'DEPENDENCIA', 'SUBDEPENDENCIA', 'PAIS', 'ESTADO', 'CVU'],
      dtype='object')

In [367]:
data.shape

(8188, 35)

# nos quedamos con el primer elemento de las siguientes columnas para generar dummies

In [368]:
columnas_esquema = ["USUARIO", "CVU", "NOMBRE","CVE_RCEA","AREA","DISCIPLINA","SUBDISCIPLINA","INSTITUCION","PAIS",
                   "ESTADO","DEPENDENCIA","SUBDEPENDENCIA","DESCRIPCION_PROYECTO","PALABRAS_CLAVE1","PALABRAS_CLAVE2","PALABRAS_CLAVE3"]

In [369]:
data_final = pd.DataFrame(columns=columnas_esquema)
data_final.head()

,USUARIO,CVU,NOMBRE,CVE_RCEA,AREA,DISCIPLINA,SUBDISCIPLINA,INSTITUCION,PAIS,ESTADO,DEPENDENCIA,SUBDEPENDENCIA,DESCRIPCION_PROYECTO,PALABRAS_CLAVE1,PALABRAS_CLAVE2,PALABRAS_CLAVE3


In [370]:
data_final = [data[data["CVE_RCEA"]==cve_rcea].reset_index(drop=True).loc[0,["USUARIO", "CVU", "NOMBRE","CVE_RCEA","AREA",
                                                                "DISCIPLINA","SUBDISCIPLINA","INSTITUCION",
                                                                "PAIS","ESTADO","DEPENDENCIA","SUBDEPENDENCIA","DESCRIPCION_PROYECTO",
                                                                "PALABRAS_CLAVE1","PALABRAS_CLAVE2","PALABRAS_CLAVE3"
                                                               ]].values for cve_rcea in data["CVE_RCEA"].unique()]

In [371]:
data_final = pd.DataFrame(data_final, columns=columnas_esquema)
data_final.head()

,USUARIO,CVU,NOMBRE,CVE_RCEA,AREA,DISCIPLINA,SUBDISCIPLINA,INSTITUCION,PAIS,ESTADO,DEPENDENCIA,SUBDEPENDENCIA,DESCRIPCION_PROYECTO,PALABRAS_CLAVE1,PALABRAS_CLAVE2,PALABRAS_CLAVE3
0,X_jpena904,904,JUAN JOSE PE?A CABRIALES,RCEA-06-06773-2002,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,MATEMATICAS,NaN,CENTRO DE INVESTIGACION Y DE ESTUDIOS AVANZADO...,MEX,DF,UNIDAD IRAPUATO,DEPARTAMENTO DE BIOTECNOLOGIA Y BIOQUIMICA,mund actual alt dinam complej condicion socioa...,Inteligencia artificial,Agricultura urbana y periurbana,Sistema de soporte de decisiones
1,X_hmorales22194,22194,HELDA ELEONORA DE GUADALUPE MORALES,RCEA-06-11606-2006,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,MATEMATICAS,NaN,EL COLEGIO DE LA FRONTERA SUR,MEXICO,CHIAPAS,UNIDAD SAN CRISTOBAL DE LAS CASAS,UNIDAD SAN CRISTOBAL DE LAS CASAS,mund actual alt dinam complej condicion socioa...,Inteligencia artificial,Agricultura urbana y periurbana,Sistema de soporte de decisiones
2,X_aperez58970,58970,ARTURO PEREZ VAZQUEZ,RCEA-06-17175-2009,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,MATEMATICAS,NaN,COLEGIO DE POSTGRADUADOS EN CIENCIAS AGRICOLAS,MEXICO,VERACRUZ,CAMPUS VERACRUZ,CAMPUS VERACRUZ,mund actual alt dinam complej condicion socioa...,Inteligencia artificial,Agricultura urbana y periurbana,Sistema de soporte de decisiones
3,X_gordonez11603,11603,GERARDO MANUEL ORDO?EZ BARBA,RCEA-05-06026-2003,CIENCIAS SOCIALES,ECONOMIA,NaN,"EL COLEGIO DE LA FRONTERA NORTE, A.C.",MEXICO,BAJA CALIFORNIA,CENTRO DE ESTUDIOS FRONTERIZOS DEL NORTE DE ME...,CENTRO DE ESTUDIOS FRONTERIZOS DEL NORTE DE ME...,polit public enfrent pobrez extrem mediant dis...,Pobreza extrema,Seguridad alimentaria,Perdidas y desperdicios de alimentos
4,X_facosta12725,12725,FELIX ACOSTA DIAZ,RCEA-05-06058-2003,CIENCIAS SOCIALES,ECONOMIA,NaN,"EL COLEGIO DE LA FRONTERA NORTE, A.C.",MEXICO,BAJA CALIFORNIA,DIRECCION GENERAL ACADEMICA,DIRECCION GENERAL ACADEMICA,polit public enfrent pobrez extrem mediant dis...,Pobreza extrema,Seguridad alimentaria,Perdidas y desperdicios de alimentos


In [372]:
data_final.shape

(3421, 16)

# Definnimos las columnas que funcionnaran como llave en el gb que concatena los textos de el evaluador

In [373]:
columnas_gb = ["USUARIO", "CVU", "NOMBRE","CVE_RCEA","AREA",
"DISCIPLINA","SUBDISCIPLINA","INSTITUCION",
"PAIS","ESTADO","DEPENDENCIA","SUBDEPENDENCIA","DESCRIPCION_PROYECTO",
"PALABRAS_CLAVE1","PALABRAS_CLAVE2","PALABRAS_CLAVE3"]

In [374]:
textos_investigador = data.groupby("CVU"
                                  )['DESCRIPCION_PROYECTO'].apply(lambda x: "%s" % ' '.join(x))

In [375]:
len(textos_investigador)

3421

## guardamos los valortes de la lista en un DF

In [376]:
df_texto_eval = pd.DataFrame()
df_texto_eval["CVU"] = textos_investigador.index
df_texto_eval["DESCRIPCION_PROYECTO_join"] = textos_investigador.values

In [377]:
df_texto_eval.shape

(3421, 2)

## Juntamos las tablas

In [378]:
df_texto_eval = pd.merge(data_final,df_texto_eval, on='CVU', how='left')

In [379]:
df_texto_eval.head(2)

,USUARIO,CVU,NOMBRE,CVE_RCEA,AREA,DISCIPLINA,SUBDISCIPLINA,INSTITUCION,PAIS,ESTADO,DEPENDENCIA,SUBDEPENDENCIA,DESCRIPCION_PROYECTO,PALABRAS_CLAVE1,PALABRAS_CLAVE2,PALABRAS_CLAVE3,DESCRIPCION_PROYECTO_join
0,X_jpena904,904,JUAN JOSE PE?A CABRIALES,RCEA-06-06773-2002,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,MATEMATICAS,NaN,CENTRO DE INVESTIGACION Y DE ESTUDIOS AVANZADO...,MEX,DF,UNIDAD IRAPUATO,DEPARTAMENTO DE BIOTECNOLOGIA Y BIOQUIMICA,mund actual alt dinam complej condicion socioa...,Inteligencia artificial,Agricultura urbana y periurbana,Sistema de soporte de decisiones,mund actual alt dinam complej condicion socioa...
1,X_hmorales22194,22194,HELDA ELEONORA DE GUADALUPE MORALES,RCEA-06-11606-2006,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,MATEMATICAS,NaN,EL COLEGIO DE LA FRONTERA SUR,MEXICO,CHIAPAS,UNIDAD SAN CRISTOBAL DE LAS CASAS,UNIDAD SAN CRISTOBAL DE LAS CASAS,mund actual alt dinam complej condicion socioa...,Inteligencia artificial,Agricultura urbana y periurbana,Sistema de soporte de decisiones,mund actual alt dinam complej condicion socioa...


In [380]:
df_texto_eval.shape

(3421, 17)

## guardamos las "palabras _clave" en una sola columna y nons deshechamos de las demás.

In [381]:
df_texto_eval["PALABRAS_CLAVE"] =  "['" + df_texto_eval["PALABRAS_CLAVE1"] + "' ,'" + df_texto_eval["PALABRAS_CLAVE2"] + "',' " + df_texto_eval["PALABRAS_CLAVE3"] + "']"
df_texto_eval.drop(["PALABRAS_CLAVE1","PALABRAS_CLAVE2","PALABRAS_CLAVE3","DESCRIPCION_PROYECTO"],axis=1,inplace=True)

### Creamos la columna nombre_apellidos:

In [382]:
df_texto_eval["APELLIDOS"] =  df_texto_eval["NOMBRE"]

In [383]:
df_texto_eval.head()

,USUARIO,CVU,NOMBRE,CVE_RCEA,AREA,DISCIPLINA,SUBDISCIPLINA,INSTITUCION,PAIS,ESTADO,DEPENDENCIA,SUBDEPENDENCIA,DESCRIPCION_PROYECTO_join,PALABRAS_CLAVE,APELLIDOS
0,X_jpena904,904,JUAN JOSE PE?A CABRIALES,RCEA-06-06773-2002,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,MATEMATICAS,NaN,CENTRO DE INVESTIGACION Y DE ESTUDIOS AVANZADO...,MEX,DF,UNIDAD IRAPUATO,DEPARTAMENTO DE BIOTECNOLOGIA Y BIOQUIMICA,mund actual alt dinam complej condicion socioa...,"['Inteligencia artificial' ,'Agricultura urban...",JUAN JOSE PE?A CABRIALES
1,X_hmorales22194,22194,HELDA ELEONORA DE GUADALUPE MORALES,RCEA-06-11606-2006,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,MATEMATICAS,NaN,EL COLEGIO DE LA FRONTERA SUR,MEXICO,CHIAPAS,UNIDAD SAN CRISTOBAL DE LAS CASAS,UNIDAD SAN CRISTOBAL DE LAS CASAS,mund actual alt dinam complej condicion socioa...,"['Inteligencia artificial' ,'Agricultura urban...",HELDA ELEONORA DE GUADALUPE MORALES
2,X_aperez58970,58970,ARTURO PEREZ VAZQUEZ,RCEA-06-17175-2009,CIENCIAS FISICO MATEMATICAS Y CIENCIAS DE LA T...,MATEMATICAS,NaN,COLEGIO DE POSTGRADUADOS EN CIENCIAS AGRICOLAS,MEXICO,VERACRUZ,CAMPUS VERACRUZ,CAMPUS VERACRUZ,mund actual alt dinam complej condicion socioa...,"['Inteligencia artificial' ,'Agricultura urban...",ARTURO PEREZ VAZQUEZ
3,X_gordonez11603,11603,GERARDO MANUEL ORDO?EZ BARBA,RCEA-05-06026-2003,CIENCIAS SOCIALES,ECONOMIA,NaN,"EL COLEGIO DE LA FRONTERA NORTE, A.C.",MEXICO,BAJA CALIFORNIA,CENTRO DE ESTUDIOS FRONTERIZOS DEL NORTE DE ME...,CENTRO DE ESTUDIOS FRONTERIZOS DEL NORTE DE ME...,polit public enfrent pobrez extrem mediant dis...,"['Pobreza extrema' ,'Seguridad alimentaria',' ...",GERARDO MANUEL ORDO?EZ BARBA
4,X_facosta12725,12725,FELIX ACOSTA DIAZ,RCEA-05-06058-2003,CIENCIAS SOCIALES,ECONOMIA,NaN,"EL COLEGIO DE LA FRONTERA NORTE, A.C.",MEXICO,BAJA CALIFORNIA,DIRECCION GENERAL ACADEMICA,DIRECCION GENERAL ACADEMICA,polit public enfrent pobrez extrem mediant dis...,"['Pobreza extrema' ,'Seguridad alimentaria',' ...",FELIX ACOSTA DIAZ


In [384]:
df_texto_eval.to_csv("./data/procesado/data_training_por_evaluador.csv",index=False)

In [385]:
df_texto_eval.shape

(3421, 15)

In [389]:
df_texto_eval[["CVU"]].to_csv("./data/procesado/cvu_only.csv",index=False)

In [35]:
df_texto_eval["len_text"] = df_texto_eval["DESCRIPCION_PROYECTO_join"].astype(str).str.split().str.len()

df_texto_eval = df_texto_eval.sort_values(by="len_text",ascending=False)

KeyError: 'DESCRIPCION_PROYECTO_join'

In [ ]:
sns.distplot(df_texto_eval["len_text"],bins=50,axlabel="#palabras",label="Textos",norm_hist=False,rug=True,kde=False,)
sns.distplot(df_texto_eval["textos_por_eval"],bins=50,axlabel="#palabras",label="Textos",norm_hist=False,rug=True,kde=False,)

In [ ]:
g = sns.jointplot(x="len_text", y="textos_por_eval", data=df_texto_eval[df_texto_eval["len_text"]<1000000], kind="kde")
g.plot_joint(plt.scatter, c="black", s=4, linewidth=0.001)
g.ax_joint.collections[0].set_alpha(0);

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2